# Import modules

In [1]:
# from dataset.speech.convert_mp3_to_wav import convert_mp3_to_wav
from dataset.dataset import CustomDataset
from dataset.text.vocabulary import PhonemeVocabv2, Vocabulary

In [2]:
import torch
from torch.utils.data import ConcatDataset, DataLoader
import torch.optim as optim

In [3]:
from model.rnnt.rnnt import RNNT

e:\UIT\NCKH\Phonological Approach for Speech to Text in Vietnamese\Source code\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [allow_tf32, disable_jit_profiling]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
INFO:speechbrain.nnet.loss.transducer_loss:Numba verbose is deactivated. To enable it, set NUMBA_VERBOSE to 1.


In [4]:
from train import Trainer

# Dataset

In [5]:
# convert_mp3_to_wav("Vietnamese-Speech-to-Text-datasets//Common-Voice")
# convert_mp3_to_wav("Vietnamese-Speech-to-Text-datasets//ViVOS")

## Load dataset

In [6]:
paths = [
    # "Vietnamese-Speech-to-Text-datasets//Common-Voice//train.json",
    # "Vietnamese-Speech-to-Text-datasets//Common-Voice//dev.json",
    # "Vietnamese-Speech-to-Text-datasets//Common-Voice//test.json",
    "Vietnamese-Speech-to-Text-datasets//ViVOS//train.json",
    "Vietnamese-Speech-to-Text-datasets//ViVOS//test.json"
]

In [7]:
SAMPLE_RATE = 16000
NUM_SAMPLE = 55000  # Chọn tạm để test (nên chọn lại)
TRANSFORMATION = "MFCC"
VOCAB = Vocabulary(paths)
MAX_LEN = VOCAB.max_len

In [8]:
# common_voice_train = CustomDataset(
#     audio_directory="Vietnamese-Speech-to-Text-datasets//Common-Voice//converted voices",
#     data_directory="Vietnamese-Speech-to-Text-datasets//Common-Voice//train.json",
#     vocab=VOCAB,
#     target_sample_rate=SAMPLE_RATE,
#     num_samples = NUM_SAMPLE,
#     transformation=TRANSFORMATION,
#     max_len=MAX_LEN
# )

# common_voice_dev = CustomDataset(
#     audio_directory="Vietnamese-Speech-to-Text-datasets//Common-Voice//converted voices",
#     data_directory="Vietnamese-Speech-to-Text-datasets//Common-Voice//dev.json",
#     vocab=VOCAB,
#     target_sample_rate=SAMPLE_RATE,
#     num_samples = NUM_SAMPLE,
#     transformation=TRANSFORMATION,
#     max_len=MAX_LEN
# )

# common_voice_test = CustomDataset(
#     audio_directory="Vietnamese-Speech-to-Text-datasets//Common-Voice//converted voices",
#     data_directory="Vietnamese-Speech-to-Text-datasets//Common-Voice//test.json",
#     vocab=VOCAB,
#     target_sample_rate=SAMPLE_RATE,
#     num_samples = NUM_SAMPLE,
#     transformation=TRANSFORMATION,
#     max_len=MAX_LEN
# )

In [9]:
vivos_train = CustomDataset(
    audio_directory="Vietnamese-Speech-to-Text-datasets//ViVOS//converted voices",
    data_directory="Vietnamese-Speech-to-Text-datasets//ViVOS//train.json",
    vocab=VOCAB,
    target_sample_rate=SAMPLE_RATE,
    num_samples = NUM_SAMPLE,
    transformation=TRANSFORMATION,
    max_len=MAX_LEN
)

vivos_test = CustomDataset(
    audio_directory="Vietnamese-Speech-to-Text-datasets//ViVOS//converted voices",
    data_directory="Vietnamese-Speech-to-Text-datasets//ViVOS//test.json",
    vocab=VOCAB,
    target_sample_rate=SAMPLE_RATE,
    num_samples = NUM_SAMPLE,
    transformation=TRANSFORMATION,
    max_len=MAX_LEN
)

## Load dataloader

In [10]:
BATCH_SIZE = 64

In [11]:
# dataset = ConcatDataset([common_voice_train, common_voice_dev, common_voice_test, vivos_train, vivos_test])

In [12]:
# train_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.8, 0.2])
# train_dataset, dev_dataset = torch.utils.data.random_split(train_dataset, [0.8, 0.2])

In [13]:
# train_loader = DataLoader(
#     train_dataset,
#     batch_size=BATCH_SIZE,
#     shuffle=True
# )
# dev_loader = DataLoader(
#     dev_dataset,
#     batch_size=BATCH_SIZE,
#     shuffle=True
# )
# test_loader = DataLoader(
#     test_dataset,
#     batch_size=BATCH_SIZE,
#     shuffle=True
# )

In [14]:
train_loader = DataLoader(
    vivos_train,
    batch_size=BATCH_SIZE,
    shuffle=True
)
dev_loader = DataLoader(
    vivos_test,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Train model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
rnnt = RNNT(
    vocab=VOCAB,
    input_size=50,
    device=device
)

In [ ]:
trainer = Trainer(model=rnnt, lr=0.0003, vocab=VOCAB, device=device)
NUM_EPOCHS = 10
train_losses = []
test_losses = []

for epoch in range(NUM_EPOCHS):
    train_loss = trainer.train(train_loader=train_loader, max_len=MAX_LEN)
    test_loss = trainer.test(dev_loader=dev_loader, max_len=MAX_LEN)
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    print("Epoch %d: train loss = %f, test loss = %f" % (epoch, train_loss, test_loss))

C:\Users\uyenl\AppData\Local\Programs\Python\Python312\Lib\inspect.py:1007: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):
  0%|          | 0/183 [00:00<?, ?it/s]